In [ ]:
### imports

# external modules
import os
import sys
import json
import glob
import tarfile as tar
import numpy as np
from fnmatch import fnmatch
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../'))
sys.path.append(topdir)

from automasking.tools.lumisection_time_parser import LumisectionTimeParser
from automasking.tools.automask_file_parsing import get_automask_from_tarfiles
from automasking.tools.automask_file_parsing import automask_to_map
import plotting.plottools as plottools

In [ ]:
# find and sort all tar files

# set input directory
inputdir = '/eos/project-m/mlplayground/public/PixelAutomasking/PixelAutoMasking/2024'

# find all .tar.xz files recursively
pattern = os.path.join(inputdir, '**/*.tar.xz')
tarfiles = glob.glob(pattern, recursive=True)
print(f'Found {len(tarfiles)} tar files in {inputdir}.')

# sort them according to date
tarfiles.sort(key=lambda x: os.path.basename(x))
for f in tarfiles[:10]: print(f'  - {f}')

In [ ]:
# make time parser instance

jsonfile = 'omsdata/omsdata_lumisection_timestamps.json'
ltp = LumisectionTimeParser(jsonfile)

In [ ]:
# get automask

tracker = 'BPix'
layer = 1

automask_info = get_automask_from_tarfiles(tarfiles, ltp=ltp, tracker=tracker, layer=layer)

In [ ]:
# write to json file

outputdir = 'automask_data'
if not os.path.exists(outputdir): os.makedirs(outputdir)
outputfile = os.path.join(outputdir, f'automask_2024_layer{layer}.json')
with open(outputfile, 'w') as f:
    json.dump(automask_info, f)

In [ ]:
# get an example

run = 379470
lumi = 91
key = str(run) + '_' + str(lumi)

automask_map = automask_to_map(automask_info[key], layer=1)
title = f'Automask for run {run}, LS {lumi}'
fig, ax = plottools.plot_hist_2d(automask_map, figsize=(12,6), title=title, titlesize=15,
                xaxtitle=None, xaxtitlesize=None, yaxtitle=None, yaxtitlesize=None,
                ticklabelsize=12, colorticklabelsize=12, extent=None, aspect=None, caxrange=(0,1),
                docolorbar=True, caxtitle='Mask', caxtitlesize=15, caxtitleoffset=15,
                origin='lower')